In [140]:
# Import dataset

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("Length of dataset in characters: ", len(text))
print(text[:1000])

Length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [141]:
# Identify all chars in the data

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [142]:
# Tokenize (character-level)

string_to_int = { c:i for i,c in enumerate(chars)}
int_to_string = { i:c for i,c in enumerate(chars)}

encode = lambda s:[string_to_int[character] for character in s]
decode = lambda l: ''.join([int_to_string[integer] for integer in l])\

# Test inputs:
#print(encode("Hello world"))
#print(decode(encode("Hello world")))


In [143]:
# Encode entire data set

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [144]:
# Split data into training and test (90-10)

n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

In [145]:
# Feed chunks from data set into transformer

chunk_size = 8
train_data[:chunk_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [146]:
'''
# Transformer parameters & predicting next character

X = train_data[:chunk_size]
y = train_data[1:chunk_size + 1]
for t in range(chunk_size):
    context = X[:t + 1]
    target = y[t]
    print(f"When input is {context}: Target: {target}")
'''

'\n# Transformer parameters & predicting next character\n\nX = train_data[:chunk_size]\ny = train_data[1:chunk_size + 1]\nfor t in range(chunk_size):\n    context = X[:t + 1]\n    target = y[t]\n    print(f"When input is {context}: Target: {target}")\n'

In [147]:
# Parallelize for multiple chunk processing (4 per run)

torch.manual_seed(1337)
batch_size = 4
chunk_size = 8

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - chunk_size, (batch_size,))
    X = torch.stack([data[i:i + chunk_size] for i in ix])
    y = torch.stack([data[i + 1:i + chunk_size + 1] for i in ix])
    return X, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)
print("----")

for k in range(batch_size):
    for j in range(chunk_size):
        context = xb[k, :j + 1]
        target = yb[k, j]
        print(f"When input is {context.tolist()}: Target: {target}")


inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is [24]: Target: 43
When input is [24, 43]: Target: 58
When input is [24, 43, 58]: Target: 5
When input is [24, 43, 58, 5]: Target: 57
When input is [24, 43, 58, 5, 57]: Target: 1
When input is [24, 43, 58, 5, 57, 1]: Target: 46
When input is [24, 43, 58, 5, 57, 1, 46]: Target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43]: Target: 39
When input is [44]: Target: 53
When input is [44, 53]: Target: 56
When input is [44, 53, 56]: Target: 1
When input is [44, 53, 56, 1]: Target: 58
When input is [44, 53, 56, 1, 58]: Target: 46
When input is [44, 53, 56, 1, 58, 46]: Target: 39
When input is [44,

In [148]:
# Feed training set into neural network (bigram LM) to make predictions

import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)    # Batch, Time, Channels tensor

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)                # Convert tensor into 2D to calculate loss
            targets = targets.view(B*T)                  # 1D target matrix
            loss = F.cross_entropy(logits, targets)     # Ideally, logits should have high number
        return logits, loss   

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)      # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)     # (B, T + 1)
        return idx                            

model = BigramLM(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [149]:
# Create optimizer using PyTorch optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [150]:
# Train model

chunk_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.458487033843994


In [151]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



KIZ! best
Whe he kee ty,

IEThoZY st:


G sayomatondgrende'TES:


Sinin: ans nd s

HRDre.
LPrer it, sof bn I's temrpeathiersptersuravu d myou ftr forgg o walos nt kit mo RLORZ'dot
Inyon! tindis prewhous oodod.
wouprebur llet if tht

Thteaix;
Malesheais whe; lino.
HIn;
CHeaimboth,
batitederour w h nig har-
ASerin livin ad:
GRveitho wond be ack'
3'd gat held inghe beromewo,
AGLL?Yos.
Wht fe, heale won:
Bergrnkit stoncetasollar dir ce s:

FGS OXELEurdo the horerthe I P,-
NENe,
LI bor d,

CUSwamadr 


In [152]:
# Self-attention (make tokens interact with one another)

torch.manual_seed(1337)
B,T,C = 4,8,2
X = torch.randn(B,T,C)
X.shape

torch.Size([4, 8, 2])

In [153]:
# Method 1: Calculate x[b,t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B,T,C))     # Bag-of-Words averaging
for b in range(B):
    for t in range(T):
        xprev = X[b, :t + 1]
        xbow[b,t] = torch.mean(xprev, 0)

print(X[0])
print(xbow[0])  # Last row is the average of all in X[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [154]:
#  Method 2: Create weights

weight = torch.tril(torch.ones(T,T))
weight = weight / weight.sum(1, keepdim=True)
xbow2 = weight @ X      # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [155]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [156]:
# Step by step analysis for method 3

weight = torch.zeros(T,T)
weight = weight.masked_fill(tril == 0, float('-inf'))   # All entries with 0, set to -ve inf
weight = F.softmax(weight, dim=-1)
weight

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [157]:
# Method 3: Use Softmax

tril = torch.tril(torch.ones(T,T))
weight = torch.zeros(T,T)
weight = weight.masked_fill(tril == 0, float('-inf'))   # All entries with 0, set to -ve inf
weight = F.softmax(weight, dim=-1)
xbow3 = weight @ X
torch.allclose(xbow, xbow3)



False

In [158]:
# Method 4: Self-attention

torch.manual_seed(1337)
B,T,C = 4,8,32
X = torch.randn(B,T,C) 

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(X)  # (B, T, 16)
q = query(X)   # (B, T, 16)
weight = q @ k.transpose(-2, -1)    # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#weight = torch.zeros((T,T))
weight = weight.masked_fill(tril == 0, float('-inf'))   # Decoder block (triangular structure)
weight = F.softmax(weight, dim=-1)

v = value(X)
out = weight @ X

out.shape
weight[0] 

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [159]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
weight = q @ k.transpose(-2,-1) * head_size**-0.5

In [160]:
k.var()


tensor(1.0449)

In [161]:
q.var()


tensor(1.0700)

In [162]:
weight.var()

tensor(1.0918)

In [163]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [164]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])